# Parkinson's Diagnose

In [7]:
import os
os.chdir('/Users/hanliyang/Documents/GitHub/Parkinson-Prediagnosis')

## Audio Conversion
* Using assembly AI

In [55]:
import assemblyai as aai

aai.settings.api_key = "d9ccd193e5e54a4ca8bdddaf3769bc59"

config = aai.TranscriptionConfig(speaker_labels=True)

transcriber = aai.Transcriber()

audioPath = ""

def audioTextConversion(audioPath):
    dialogue = {
        "Speaker A" : [],
        "Speaker B" : []
    }
    transcript = transcriber.transcribe(
    audioPath,
    config=config
    )

    for utterance in transcript.utterances:
        #print(f"Speaker {utterance.speaker}: {utterance.text}")
        speaker = f"Speaker {utterance.speaker}"
        dialogue[speaker].append(utterance.text)
    return speaker_differentiation(dialogue)
    
def speaker_differentiation(dialogue):
    totalA = 0
    totalB = 0
    for text in dialogue['Speaker A']:
        totalA += len(text)
    
    for text in dialogue['Speaker B']:
        totalB += len(text)

    if totalA > totalB:
        return {'participant':' '.join(dialogue["Speaker A"]), 
                "interviewer": ' '.join(dialogue["Speaker B"])} 
    else:
        return {'participant':' '.join(dialogue["Speaker B"]), 
                "interviewer": ' '.join(dialogue["Speaker A"])} 
 

In [54]:
import pandas as pd
dialogue = audioTextConversion('NLP/Data/SpontaneousDialogue/HC/ID00_hc_0_0_0.wav')

#df = pd.DataFrame(data = dialogue)


Speaker A: Okay, great. Thank you.
Speaker B: Thank you.
Speaker A: I would try the spontaneous dialogue. So I will start with a question. For example, it's my first time in London and could you recommend some place nice places I should visit?
Speaker B: St. James's park in London. And Buckingham palace and. Sorry. Now what other places? Trafalgar Square. Trafalgar Square. And.
Speaker C: Buckingham Palace.


KeyError: 'Speaker C'

In [22]:
dialogue

{'participant': "Hi. Yeah. In London, you can go to Oxford street, which is famous for shopping, and there's selfishes there, and a lot of tourists come there and a lot there. So it's a good place to see when you come to London. And there's Buckingham palace, where the Queen lives, and that's a good place, the way the royal family lives. So you can come there. And there's other big Ben houses of parliament, where the government. The government are british government are. So that's good places to go. And then there's the London Eye, where you can see the whole of London. You go and you can see the whole of London. So that's good places to visit in London. Yeah. So if I was. I was coming, would you recommend. I think, yeah, I would go by public transport. Then you would see how London really is, and you see the whole of London. If you go on a bus or the transport, you would see how London is. So I would advise to go on public transport, buses and trains, so you'd really enjoy it. And you

## Counting Pos

In [11]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

tokens = "Computer science is the study of computation, automation, and information."\
"Computer science spans theoretical disciplines (such as algorithms, theory of computation, information theory, and automation) to practical disciplines (including the design and implementation of hardware and software)."\
"Computer science is generally considered an academic discipline and distinct from computer programming."

[nltk_data] Downloading package punkt to /Users/hanliyang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hanliyang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


* verbs - VB	verb (ask)
VBG	verb gerund (judging)
VBD	verb past tense (pleaded)
VBN	verb past participle (reunified)
VBP	verb, present tense not 3rd person singular(wrap)
VBZ	verb, present tense with 3rd person singular (bases)
* common nouns - NN, NNS
* proper nouns - NNP, NNPS


In [30]:
filler_words = [
    "um", "uh", "er", "like", "you know", "i mean", "well", "so", "okay", "right",
    "actually", "basically", "literally", "kind of", "sort of", "anyway", "or something",
    "whatever", "i guess", "you see", "got it", "i suppose", "just", "yeah", "yep",
    "seriously", "alright", "hmm", "look", "totally", "alright then", "know what i mean?",
    "at the end of the day", "to be honest", "if you know what i mean"
]

def pdFeaturesCount(tokens):
  wordList = pos_tag(word_tokenize(tokens))
  totalWords, verbsRate, verbs, commonNouns, properNouns, fillers = 0, 0, 0, 0, 0, 0
  for word, pos in wordList:
    #print(word, pos)
    totalWords += 1
    if pos in ['VB', 'VBG', 'VBD', 'VBN', 'VBP', 'VBZ']:
      verbs += 1
    if pos in ['NN', 'NNS']:
      commonNouns += 1
    if pos in ['NNP', 'NNPS']:
      properNouns += 1
    if word in filler_words:
      fillers += 1

  verbsRate = round(verbs / totalWords, 3)
  commonNounsRate = round(commonNouns / totalWords, 3)
  properNounsRate = round(properNouns / totalWords, 3)
  fillersRate = round(fillers / totalWords, 3)

  return {'Total Words': totalWords, 'Vb Rate': verbsRate, 'Verbs': verbs, 'CmN rate': commonNounsRate, 'Common Nouns': commonNouns, 'PpN Rate': properNounsRate, 'Proper Nouns': properNouns, 'FlR': fillersRate, 'Filler Words': fillers}

In [32]:
dialogue.update(pdFeaturesCount(dialogue['participant']))
dialogue

{'participant': "Hi. Yeah. In London, you can go to Oxford street, which is famous for shopping, and there's selfishes there, and a lot of tourists come there and a lot there. So it's a good place to see when you come to London. And there's Buckingham palace, where the Queen lives, and that's a good place, the way the royal family lives. So you can come there. And there's other big Ben houses of parliament, where the government. The government are british government are. So that's good places to go. And then there's the London Eye, where you can see the whole of London. You go and you can see the whole of London. So that's good places to visit in London. Yeah. So if I was. I was coming, would you recommend. I think, yeah, I would go by public transport. Then you would see how London really is, and you see the whole of London. If you go on a bus or the transport, you would see how London is. So I would advise to go on public transport, buses and trains, so you'd really enjoy it. And you

## Iterating through the actual audios
* 0 -- HC
* 1 -- PD

In [61]:
import glob

data = []

for path in glob.glob('NLP/Data/SpontaneousDialogue/PD/*'):
    dialogue = {}
    dialogue['target'] = 1
    dialogue['Audio path'] = path
    print(path)
    dialogue.update(audioTextConversion(path))
    dialogue.update(pdFeaturesCount(dialogue['participant']))
    data.append(dialogue)

#31 HC not passed

NLP/Data/SpontaneousDialogue/PD/ID32_pd_3_1_1.wav
NLP/Data/SpontaneousDialogue/PD/ID30_pd_2_1_1.wav
NLP/Data/SpontaneousDialogue/PD/ID06_pd_3_1_1.wav
NLP/Data/SpontaneousDialogue/PD/ID27_pd_4_1_1.wav
NLP/Data/SpontaneousDialogue/PD/ID24_pd_2_0_0.wav
NLP/Data/SpontaneousDialogue/PD/ID13_pd_3_2_2.wav
NLP/Data/SpontaneousDialogue/PD/ID29_pd_3_1_2.wav
NLP/Data/SpontaneousDialogue/PD/ID07_pd_2_0_0.wav


In [49]:
for i in range(len(data)):
    print(data[i]['Audio path'])

NLP/Data/SpontaneousDialogue/HC/ID01_hc_0_0_0.wav
NLP/Data/SpontaneousDialogue/HC/ID14_hc_0_0_0.wav
NLP/Data/SpontaneousDialogue/HC/ID25_hc_0_0_0.wav
NLP/Data/SpontaneousDialogue/HC/ID12_hc_0_0_0.wav
NLP/Data/SpontaneousDialogue/HC/ID15_hc_0_0_0.wav
NLP/Data/SpontaneousDialogue/HC/ID00_hc_0_0_0.wav
NLP/Data/SpontaneousDialogue/HC/ID36_hc_0_0_0.wav
NLP/Data/SpontaneousDialogue/HC/ID23_hc_0_0_0.wav
NLP/Data/SpontaneousDialogue/HC/ID28_hc_0_0_0.wav


In [62]:
import pandas as pd

df = pd.DataFrame(data).reset_index(drop=True)
df1 = pd.read_csv('NLP/Data/SpontaneousDialogue/PD_Data.csv')
df = pd.concat([df,df1], axis=0)
df.to_csv('NLP/Data/SpontaneousDialogue/PD_Data.csv',index=False)